In [1]:
def sigma(self, n, chow=False, formal=False, deform=False, return_polynomial=False, check=True):
    from sage.rings.fraction_field import (FractionField, is_FractionField, FractionField_1poly_field)
    dom = self.domain()
    base_ring = self.base_ring()
    d = self.degree()
    N = dom.dimension_relative()
    f = copy(self)
    Fn = f.nth_iterate_map(n)
    CR = f.codomain().ambient_space().coordinate_ring()
    if deform:
        # we need a model with all affine periodic points
        new_f = f.affine_preperiodic_model(0, n)
        new_f.normalize_coordinates()
        # we now deform by a parameter t
        T = base_ring['k']
        k = T.gens()[0]
        Pt = ProjectiveSpace(N, R=T, names = [str(i) for i in CR.gens()])
        deformed_polys = [poly + k*Pt.gens()[-1]**d for poly in new_f.defining_polynomials()[:-1]]
        deformed_polys += [new_f.defining_polynomials()[-1]]
        f_deformed = DynamicalSystem(deformed_polys)
        sigma_poly = sigma(f_deformed, n, chow=chow, deform=False, return_polynomial=True)
        sigma_polynomial = sigma_poly.specialization({k:0})
        # we fix the ordering of the parent polynomial ring
        new_parent = sigma_polynomial.parent().change_ring(order='lex')
        sigma_polynomial = new_parent(sigma_polynomial)
        sigma_polynomial *= sigma_polynomial.coefficients()[0].inverse_of_unit()
    else:
        if not base_ring.is_field():
            F = FractionField(base_ring)
            f.normalize_coordinates()
            X = f.periodic_points(n, minimal=False, formal=formal, return_scheme=True)
            X = X.change_ring(F)
        else:
            F = base_ring
            if is_FractionField(base_ring):
                if is_MPolynomialRing(base_ring.ring()) or is_PolynomialRing(base_ring.ring()):
                    f.normalize_coordinates()
                    f_ring = f.change_ring(base_ring.ring())
                    X = f_ring.periodic_points(n, minimal=False, formal=formal, return_scheme=True)
                    X = X.change_ring(F)
            else:
                X = f.periodic_points(n, minimal=False, formal=formal, return_scheme=True)
        newR = PolynomialRing(F, 'w, t', 2, order='lex')
        if not base_ring.is_field():
            ringR = PolynomialRing(base_ring, 'w, t', 2, order='lex')
        if chow:
            # create full polynomial ring
            R = PolynomialRing(F, 'v', 2*N+3, order='lex')
            var = list(R.gens())
            # create polynomial ring for result
            R2 = PolynomialRing(F, var[:N] + var[-2:])
            psi = R2.hom(N*[0]+list(newR.gens()), newR)
            # create substition to set extra variables to 0
            R_zero = {R.gen(N):1}
            for j in range(N+1, 2*N+1):
                R_zero[R.gen(j)] = 0
            t = var.pop()
            w = var.pop()
            var = var[:N]
        else:
            R = PolynomialRing(F, 'v', N+2, order='lex')
            psi = R.hom(N*[0] + list(newR.gens()), newR)
            var = list(R.gens())
            t = var.pop()
            w = var.pop()
        sigma_polynomial = 1
        # go through each affine patch to avoid repeating periodic points
        # setting the visited coordiantes to 0 as we go
        for j in range(N,-1,-1):
            Xa = X.affine_patch(j)
            fa = Fn.dehomogenize(j)
            Pa = fa.domain()
            Ra = Pa.coordinate_ring()
            # create the images for the Hom to the ring we will do the elimination over
            # with done affine patch coordinates as 0
            if chow:
                im = [R.gen(i) for i in range(j)] + (N-j)*[0] + [R.gen(i) for i in range(N, R.ngens())]
            else:
                im = list(R.gens())[:j] + (N-j)*[0] + [R.gen(i) for i in range(N, R.ngens())]
            phi = Ra.hom(R.gens()[0:len(Ra.gens())])
            # create polymomial that evaluates to the characteristic polynomial
            M = t*matrix.identity(R, N)
            g = (M-jacobian([phi(F.numerator())/phi(F.denominator()) for F in fa], var)).det()
            # create the terms of the sigma invariants prod(w-lambda)
            g_prime = w*R(g.denominator())(im)-R(g.numerator())(im)
            # move the defining polynomials to the polynomial ring
            L = [phi(h)(im) for h in Xa.defining_polynomials()]
            # add the appropriate final polynomial to compute the sigma invariant polynomial
            # via a Poisson product in elimination
            if chow:
                L += [g_prime + sum(R.gen(j-1)*R.gen(N+j)*(R(g.denominator())(im)) for j in range(1,N+1))]
            else:
                L += [g_prime]
            I = R.ideal(L)
            # since R is lex ordering, this is an elimination step
            G = I.groebner_basis()
            # the polynomial we need is the one just in w and t
            if chow:
                poly = psi(G[-1].specialization(R_zero))
                if len(list(poly)) > 0:
                    poly *= poly.coefficients()[0].inverse_of_unit()
            else:
                poly = psi(G[-1])
            if not base_ring.is_field():
                denom = lcm([coeff[0].denominator() for coeff in poly])
                poly *= denom
            sigma_polynomial *= poly
        if not base_ring.is_field():
            sigma_polynomial = ringR(sigma_polynomial)
    if check:
        degree_w = sigma_polynomial.degrees()[0]
        expected_degree = sum(d**(n*i) for i in range(N+1))
        if degree_w != expected_degree:
            raise ValueError('sigma polynomial dropped degree, as multiplicities were not accounted for correctly. ' +
                            'try setting chow=True or deform=True')
    if return_polynomial:
        return sigma_polynomial
    # if we are returing a numerical list, read off the coefficients
    # in order of degree adjusting sign appropriately
    sigmas = []
    sigma_dictionary = dict([list(reversed(i)) for i in list(sigma_polynomial)])
    w, t = sigma_polynomial.variables()
    for i in range(degree_w+1):
        for j in range(degree_t+1):
            sigmas.append((-1)**(i+j)*sigma_dictionary.pop(w**(degree_w - i)*t**(degree_t - j), 0))
    return sigmas

In [2]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^2+3*y^2,x*y])
poly = sigma(f, 1, return_polynomial=True, check=False)
poly

w - t + 1

In [3]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2-z^2,y^2,z^2])
sigma(f, 2,  return_polynomial=True,formal = False)

Interrupting Singular...


SingularError: Singular error:
   ? unknown option `set`
   ? unknown option `sage113`
   ? error occurred in or before STDIN line 11: `option(set,sage113);`

In [2]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2-z^2,y^2,z^2])
f.sigma_invariants(1, chow=True,return_polynomial=False)

[1,
 7,
 10,
 4,
 21,
 60,
 52,
 8,
 -16,
 35,
 150,
 200,
 48,
 -112,
 -64,
 0,
 35,
 200,
 360,
 112,
 -336,
 -288,
 0,
 0,
 0,
 21,
 150,
 340,
 128,
 -496,
 -512,
 64,
 128,
 0,
 0,
 0,
 7,
 60,
 164,
 72,
 -352,
 -416,
 128,
 256,
 0,
 0,
 0,
 0,
 0,
 1,
 10,
 32,
 16,
 -96,
 -128,
 64,
 128,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [18]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2,z^2,y^2])
poly = f.sigma_invariants(1, chow=True,return_polynomial=True)
dict([reversed(tup) for tup in poly])

{w^7: 1,
 w^6*t^2: -7,
 w^6*t: -6,
 w^6: 12,
 w^5*t^4: 21,
 w^5*t^3: 36,
 w^5*t^2: -60,
 w^5*t: -72,
 w^5: 48,
 w^4*t^6: -35,
 w^4*t^5: -90,
 w^4*t^4: 120,
 w^4*t^3: 352,
 w^4*t^2: -96,
 w^4*t: -288,
 w^4: 64,
 w^3*t^8: 35,
 w^3*t^7: 120,
 w^3*t^6: -120,
 w^3*t^5: -688,
 w^3*t^4: -96,
 w^3*t^3: 1056,
 w^3*t^2: 320,
 w^3*t: -384,
 w^2*t^10: -21,
 w^2*t^9: -90,
 w^2*t^8: 60,
 w^2*t^7: 672,
 w^2*t^6: 384,
 w^2*t^5: -1440,
 w^2*t^4: -1344,
 w^2*t^3: 768,
 w^2*t^2: 768,
 w*t^12: 7,
 w*t^11: 36,
 w*t^10: -12,
 w*t^9: -328,
 w*t^8: -336,
 w*t^7: 864,
 w*t^6: 1472,
 w*t^5: -384,
 w*t^4: -1536,
 w*t^3: -512,
 t^14: -1,
 t^13: -6,
 t^11: 64,
 t^10: 96,
 t^9: -192,
 t^8: -512,
 t^6: 768,
 t^5: 512}

In [20]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2,y^2,z^2])
poly = f.sigma_invariants(1, chow=True,return_polynomial=True)
dict([reversed(tup) for tup in poly])

{w^7: 1,
 w^6*t^2: -7,
 w^6*t: 10,
 w^6: -4,
 w^5*t^4: 21,
 w^5*t^3: -60,
 w^5*t^2: 60,
 w^5*t: -24,
 w^4*t^6: -35,
 w^4*t^5: 150,
 w^4*t^4: -240,
 w^4*t^3: 176,
 w^4*t^2: -48,
 w^3*t^8: 35,
 w^3*t^7: -200,
 w^3*t^6: 440,
 w^3*t^5: -464,
 w^3*t^4: 224,
 w^3*t^3: -32,
 w^2*t^10: -21,
 w^2*t^9: 150,
 w^2*t^8: -420,
 w^2*t^7: 576,
 w^2*t^6: -384,
 w^2*t^5: 96,
 w*t^12: 7,
 w*t^11: -60,
 w*t^10: 204,
 w*t^9: -344,
 w*t^8: 288,
 w*t^7: -96,
 t^14: -1,
 t^13: 10,
 t^12: -40,
 t^11: 80,
 t^10: -80,
 t^9: 32}

In [3]:
sage: P.<x,y,z,w> = ProjectiveSpace(QQ, 3)
            sage: f = DynamicalSystem_projective([x^2, w^2, z^2, y^2])
            sage: f.sigma_invariants(1, chow=True, return_polynomial=True)

KeyboardInterrupt: 

In [6]:
sage: P.<x,y,z> = ProjectiveSpace(GF(5), 2)
            sage: f = DynamicalSystem([x^2 - z^2, y^2, z^2])
            sage: f.sigma_invariants(1, chow=True, return_polynomial=True)

w^7 - 2*w^6*t^2 + w^6 + w^5*t^4 + 2*w^5*t^2 + 2*w^5*t - w^5 - 2*w^4*t^3 + 2*w^4*t^2 + w^4*t - 2*w^3*t^5 - w^3*t^4 - 2*w^3*t^3 - w^2*t^10 - 2*w^2*t^7 + w^2*t^6 - 2*w^2*t^5 + w^2*t^4 - 2*w^2*t^3 + 2*w*t^12 - w*t^10 - 2*w*t^9 - 2*w*t^8 + w*t^7 - 2*w*t^6 - w*t^5 - t^14 - 2*t^12 + t^11 + t^10 + 2*t^9 + t^8 - 2*t^7

In [7]:
sage: R.<c,d> = QQ[]
            sage: P.<x,y,z> = ProjectiveSpace(R, 2)
            sage: f = DynamicalSystem([x^2 + c*z^2, y^2 + d*z^2, z^2])
            sage: len(dict(f.sigma_invariants(1, return_polynomial=True)))

51

In [9]:
P.<x,y> = ProjectiveSpace(QQ, 1)
f = DynamicalSystem([x^2 + 3*y^2, x*y])
f.sigma_invariants(1, deform = True, return_polynomial=True)

w^3 - 3*w^2*t + 3*w^2 + 3*w*t^2 - 6*w*t + 3*w - t^3 + 3*t^2 - 3*t + 1

In [15]:
sage: R.<w> = QQ[]
            sage: N.<n> = NumberField(w^2 + 1)
            sage: P.<x,y,z> = ProjectiveSpace(N, 2)
            sage: f = DynamicalSystem_projective([x^2, y^2, z^2])
f.sigma_invariants(1, chow=True) == f.change_ring(QQ).sigma_invariants(1, chow=True)

True

In [21]:
sage: P.<x,y,z> = ProjectiveSpace(QQ, 2)
            sage: f = DynamicalSystem_projective([x^2, z^2, y^2])
            sage: f.sigma_invariants(1, chow=True)

[1,
 7,
 -6,
 -12,
 21,
 -36,
 -60,
 72,
 48,
 35,
 -90,
 -120,
 352,
 96,
 -288,
 -64,
 35,
 -120,
 -120,
 688,
 -96,
 -1056,
 320,
 384,
 0,
 21,
 -90,
 -60,
 672,
 -384,
 -1440,
 1344,
 768,
 -768,
 0,
 0,
 7,
 -36,
 -12,
 328,
 -336,
 -864,
 1472,
 384,
 -1536,
 512,
 0,
 0,
 0,
 1,
 -6,
 0,
 64,
 -96,
 -192,
 512,
 0,
 -768,
 512,
 0,
 0,
 0,
 0,
 0]

In [1]:
sage: P.<x,y> = ProjectiveSpace(QQ, 1)
sage: f = DynamicalSystem([x^2 + c*y^2, y^2])
sage: f.sigma_invariants(2, chow=True, formal=True, return_polynomial=True)

w^2 + (-2)*w*t + (8*c + 8)*w + t^2 + (-8*c - 8)*t + 16*c^2 + 32*c + 16

In [ ]:
f.